In [21]:
import numpy as np
import pandas as pd

In [22]:
data = pd.read_csv("/content/drive/MyDrive/mydata/project/en_wiki_small.csv")
data.head()
# col_count = 10
epochs = 200
cols = list(data.columns)
sites = cols[1:]
data = data[sites]
data.head()

,"""Weird_Al""_Yankovic_en.wikipedia.org_desktop_all-agents",1989_(Taylor_Swift_album)_en.wikipedia.org_desktop_all-agents,1918_flu_pandemic_en.wikipedia.org_desktop_all-agents,2020_Summer_Olympics_en.wikipedia.org_desktop_all-agents,AR-15_en.wikipedia.org_desktop_all-agents,Academy_Award_for_Best_Actor_en.wikipedia.org_desktop_all-agents,Academy_Award_for_Best_Picture_en.wikipedia.org_desktop_all-agents,2003_invasion_of_Iraq_en.wikipedia.org_desktop_all-agents,Alex_Jones_(radio_host)_en.wikipedia.org_desktop_all-agents,Akshay_Kumar_en.wikipedia.org_desktop_all-agents
0,2403.0,4768.0,868.0,1931.0,1753.0,1450.0,3077.0,1153.0,1383.0,2536.0
1,20136.0,4346.0,877.0,1592.0,1603.0,1380.0,3157.0,1055.0,966.0,2482.0
2,1850.0,4251.0,688.0,1273.0,1736.0,1356.0,2726.0,987.0,865.0,2443.0
3,1432.0,3815.0,623.0,1422.0,1335.0,1413.0,3275.0,880.0,812.0,2432.0
4,1351.0,3970.0,604.0,1797.0,1399.0,1507.0,3379.0,885.0,863.0,2588.0


In [23]:
def split_seq_c(D, inp_size,test_size=100):
  start_indices = np.array(range(0,len(D)-test_size-inp_size))
  return [
    np.array(list(map(lambda i:D[i:i+inp_size], start_indices))),
    np.array(list(map(lambda i:D[i+inp_size], start_indices))),
    np.array(D[-test_size:]),
  ]

sequences = []
n_steps = 10
for site in sites:
    X, y, test_data = split_seq_c(data[site], n_steps)
    sequences.append({
        'X': X,
        'y': y,
        'test_data':test_data,
    })
len(sequences)

10

In [24]:
# for seq in sequences:
#     print('\n'.join([f'{x}: {y_}' for x,y_ in zip(seq['X'],seq['y'])]))

# seq = sequences[0]
# print('\n'.join([f'{x}: {y_}' for x,y_ in zip(seq['X'],seq['y'])]))

In [25]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional

BiDirectional LSTM

In [26]:
vanillas = []

for seq in sequences:
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    vanillas.append(model)


In [27]:
for i in range(len(vanillas)):
    X = seq['X'].reshape(list(seq['X'].shape)+[1])
    vanillas[i].fit(X, seq['y'], epochs=epochs, verbose=0)
    print(f'{i+1} done')

1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done


In [28]:
def predict(model, x):
    x_inp = np.array(x)
    x_inp = x_inp.reshape(1,n_steps, 1)
    return model.predict(x_inp, verbose=0).reshape([])
    
def internal_error(model, X, y):
    y_pred = np.array([predict(model, x) for x in X])
    y_true = np.array(y)
    return np.mean(np.abs(y_true-y_pred))

# errors = []
# for i in range(len(sites)):
#     errors.append(internal_error(vanillas[i], sequences[i]['X'], sequences[i]['y']))
#     print(f"Error for site {i+1} : {errors[-1]}")

# print()
# print(np.sum(errors)/len(errors))

In [29]:
def external_error(model, last_x, test_data, verbose=0):
    x = last_x

    error_sum = 0
    percent_error_sum = 0
    for i in range(len(test_data)):
        y = predict(model, x)
        error_sum += abs(y-test_data[i])
        percent_error_sum += 100*abs(y - test_data[i])/test_data[i]
        if verbose:
            print(
                y, "\t",
                test_data[i], "\t",
                abs(y - test_data[i]), "\t",
                100*abs(y - test_data[i])/test_data[i]
            )
        x = np.array(list(x[1:]) + [sequences[0]['test_data'][i]])
    return {
        "MAE": error_sum/len(test_data),
        "MAPE": percent_error_sum/len(test_data),
    }

errors = []
for i in range(len(sites)):
    errors.append(external_error(vanillas[i], sequences[i]['X'][-1], sequences[i]['test_data']))
    print(f"Site {i+1} :\nerror: {errors[-1]['MAE']}\tmaean value: {np.mean(sequences[i]['test_data'])}\tpercentage error: {errors[-1]['MAPE']}")



Site 1 :
error: 336.28834228515626	maean value: 1704.37	percentage error: 17.131348247520535
Site 2 :
error: 446.54749084472655	maean value: 1237.95	percentage error: 36.096139904632196
Site 3 :
error: 2434.3140728759768	maean value: 3320.87	percentage error: 42.646140385327335
Site 4 :
error: 430.9263592529297	maean value: 1320.23	percentage error: 36.089148425497406
Site 5 :
error: 330.72614318847656	maean value: 1544.94	percentage error: 21.56666701443216
Site 6 :
error: 390.8462109375	maean value: 1683.9	percentage error: 23.07556477226137
Site 7 :
error: 1642.1525317382811	maean value: 3315.98	percentage error: 48.04856243798442
Site 8 :
error: 413.20606323242185	maean value: 1617.99	percentage error: 25.67346604212608
Site 9 :
error: 1568.4328881835938	maean value: 3137.74	percentage error: 42.15432696434995
Site 10 :
error: 1028.7693811035156	maean value: 2676.69	percentage error: 35.66768606526247


In [30]:
np.mean(sequences[1]['test_data'])

1237.95